In [1]:
import numpy as np
from scipy.linalg import eig

In [11]:
# define the parameters for the system

E = 0.0
t = -1.0
v = 0.0
N = 5
M = 2

# set up the lead matrix  
H = np.diag(v * np.ones(N) - E) + np.diag(t * np.ones(N - 1), k=1) + np.diag(t * np.ones(N - 1), k = -1)
I = np.diag(np.ones(N))
V = np.diag(t * np.ones(N))
Null = np.zeros((N, N))
A = np.stack((np.stack((Null, I), axis=1), np.stack((V, H), axis=1))).reshape(2*N, 2*N)
B = np.stack((np.stack((I, Null), axis=1), np.stack((Null, V), axis=1))).reshape(2*N, 2*N)
print(A)
print(B)

[[ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [-1.  0.  0.  0.  0.  0. -1.  0.  0.  0.]
 [ 0. -1.  0.  0.  0. -1.  0. -1.  0.  0.]
 [ 0.  0. -1.  0.  0.  0. -1.  0. -1.  0.]
 [ 0.  0.  0. -1.  0.  0.  0. -1.  0. -1.]
 [ 0.  0.  0.  0. -1.  0.  0.  0. -1.  0.]]
[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. -1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. -1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0. -1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0. -1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0. -1.]]


In [12]:
# diagonalize the lead matrix
w, v = eig(A, B)
print(w)

[-2.18890106+0.j  2.18890106+0.j -1.61803399+0.j  1.61803399+0.j
 -1.        +0.j -0.61803399+0.j -0.45685025+0.j  1.        +0.j
  0.61803399+0.j  0.45685025+0.j]


In [13]:
# order the modes based on propagating, left/right moving
modes = [vec[:2] for vec in v.T]
self_energy_left = 0.
self_energy_right = 0.
V_L = np.array([[t, 0.], [0., t]])
V_R = np.array([[t, 0.], [0., t]])

org_modes = {'evan_R': [], 'evan_L': [], 'prop_R': [], 'prop_L': []}
for e, m in zip(w, modes):
    print(e)
    total = 0.
    for i in range(1,N//2):
        total += m[i].conjugate() * -t * m[i-1]
    if np.absolute(e) < 1:
        if total < 0:
            org_modes['evan_L'].append(m)
        else:
            org_modes['evan_R'].append(m)
    else:
        if total < 1:
            org_modes['prop_R'].append(m)
        else:
            org_modes['prop_L'].append(m)

print(org_modes)
for i, m in enumerate(modes):
    self_energy_left += w[i]**(-1) * np.outer(m, m.conjugate())
    self_energy_right += w[i] * np.outer(m, m.conjugate())
self_energy_left *= V_L
self_energy_right *= V_R

(-2.188901059316734+0j)
(2.1889010593167364+0j)
(-1.6180339887498965+0j)
(1.6180339887498967+0j)
(-1.000000000000002+0j)
(-0.6180339887498959+0j)
(-0.4568502517478575+0j)
(1.0000000000000009+0j)
(0.6180339887498956+0j)
(0.4568502517478571+0j)
{'evan_R': [array([0.4253254, 0.4253254]), array([-0.26257171, -0.45478754])], 'evan_L': [array([-0.4253254,  0.4253254]), array([ 0.26257171, -0.45478754])], 'prop_R': [array([-0.11995595,  0.2077698 ]), array([-0.11995595, -0.2077698 ]), array([ 0.26286556, -0.26286556]), array([-0.26286556, -0.26286556]), array([ 4.08248290e-01, -3.15748051e-17]), array([-4.08248290e-01,  6.37217707e-16])], 'prop_L': []}


In [ ]:
rows = [None] * M

for i in range(M):
    if i == 0:
        rows[i] = H + self_energy_left
    elif i == 1:
        rows[i] = V
    else:
        rows[i] = Null
    
for i in range(M):
    for j in range(1,M):
        if i == j:
            if j == M - 1:
                rows[i] = np.stack((rows[i], H + self_energy_right), axis=1).reshape((j + 1)*N, N)
            else:
                rows[i] = np.stack((rows[i], H), axis=1).reshape((j + 1)*N, N)
        if i == j - 1 or i == j + 1:
            rows[i] = np.stack((rows[i], V), axis=1).reshape((j + 1)*N, N)

total_H = np.stack((rows[0], rows[1])).reshape(N*M ,N*M)
psi = np.linalg.solve(total_H, np.ones(2 * N))
print(psi)